In [12]:
import random
import numpy as np

In [13]:
class Network(object):
    def __init__(self,sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(n,1) for n in sizes[1:]]
        self.weights = [np.random.randn(m,n) for n,m in zip(sizes[:self.num_layers - 1],sizes[1:])]



In [14]:
nn = Network([2,3,1])


In [15]:
nn.weights

[array([[ 0.17992625, -1.96838254],
        [ 0.61243935, -1.20246901],
        [ 0.06117298,  0.90819957]]),
 array([[ 1.19477084, -1.82689987, -0.2335041 ]])]